In [1]:
!pip3 install PyGithub requests

     |████████████████████████████████| 296kB 5.4MB/s 
     |████████████████████████████████| 962kB 18.7MB/s 


In [2]:
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm
import base64
from github import Github
from io import BytesIO
import requests

In [3]:
# Username of your GitHub account
 
username = ''
 
# Personal Access Token (PAO) from your GitHub account
 
token = ''
 
# Creates a re-usable session object with your creds in-built
 
github_session = requests.Session()
github_session.auth = (username, token)

In [4]:
def download_data (url):

    url = url # Make sure the url is the raw version of the file on GitHub
    download = github_session.get(url).content

    df= pd.read_csv(BytesIO(download))
    
    return df

In [5]:
url = 'https://raw.githubusercontent.com/OmdenaAI/WeoWater/main/task-1-scraping-twitter/Twitter_Data_twint.csv'
twitter_data_twint= download_data(url)

In [6]:
twitter_data_twint.isnull()

,id,date,time,timezone,user_id,username,place,tweet,hashtags,reply_to,length,Cleaned Tweets
0,False,False,False,False,False,False,True,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
30083,False,False,False,False,False,False,True,False,False,False,False,False
30084,False,False,False,False,False,False,True,False,False,False,False,False
30085,False,False,False,False,False,False,True,False,False,False,False,False
30086,False,False,False,False,False,False,True,False,False,False,False,False


In [7]:
#Fill null values in Place column
twitter_data_twint['place'].fillna('Not Specified',inplace=True)

In [8]:
#Clean the hashtag column in proper format
tags=[]
for hash_tags in twitter_data_twint['hashtags']:
  hash_tags = hash_tags.replace("'","")
  tags.append(hash_tags.replace("[","").replace("]",""))
tags = ["NoTags" if x == '' else x for x in tags]
twitter_data_twint['hashtags']= tags

In [9]:
twitter_data_twint['hashtags'].unique()

array(['NoTags', 'tadd', 'ketsana', ...,
       'deluge, mumbairains, inundation', 'chswx, chsnews',
       'weatherupdate, isaias'], dtype=object)

In [10]:
twitter_data_twint['date/time range']= twitter_data_twint['date']+ ' ' + twitter_data_twint['time']

In [11]:
first_column = twitter_data_twint.pop('date/time range')
twitter_data_twint.insert(0,'date/time range', first_column)

In [12]:
#Droping unwanted columns 
twitter_data_twint= twitter_data_twint.drop(['id','length','reply_to','date','time'],axis=1)

In [15]:
#tagged the data with new label- flood,drought, None
category=[]
for tweet in twitter_data_twint['Cleaned Tweets']:
  if 'flood' in tweet:
    category.append('flood')
  elif any(x in tweet for x in ['flood','drought']):
    category.append('flood_and_drought')
  elif 'drought' in tweet:
    category.append('drought')
  else:
    category.append('None')

In [18]:
#Added new column category- which have new tagged label
twitter_data_twint['category']= category

In [19]:
twitter_data_twint['category'].unique()

array(['flood', 'None', 'flood_and_drought'], dtype=object)

In [20]:
twitter_data_twint.to_csv('Preprocessed_twitter_data_twint.csv')

In [21]:
twitter_data_twint

,date/time range,timezone,user_id,username,place,tweet,hashtags,Cleaned Tweets,category
0,2020-10-31 07:56:37,800,17580230,jarallen,Not Specified,@austinperroux @NWSSanAntonio Same here. I’ve ...,NoTags,"['austinperroux', 'NWSSanAntonio', 'I’ve', 'ne...",flood
1,2020-10-31 07:49:35,800,472122299,philippinestar,Not Specified,"Flooding (including flash floods), rain-induce...",NoTags,"['Flooding', 'including', 'flash', 'floods', '...",flood
2,2020-10-31 07:23:57,800,2453732882,ttweathercenter,Not Specified,7:20 PM - Flash flooding continues in Arouca t...,NoTags,"['720', 'PM', 'Flash', 'flooding', 'continues'...",flood
3,2020-10-31 07:12:06,800,2453732882,ttweathercenter,Not Specified,Another video of the flash flooding ongoing at...,NoTags,"['Another', 'video', 'flash', 'flooding', 'ong...",flood
4,2020-10-31 07:10:16,800,2453732882,ttweathercenter,Not Specified,7:05 PM - Flash flooding ongoing at Henry Stre...,NoTags,"['705', 'PM', 'Flash', 'flooding', 'ongoing', ...",flood
...,...,...,...,...,...,...,...,...,...
30083,2020-08-03 17:47:10,800,586253516,twuerzer,Not Specified,RT -- NEW: NHC_Atlantic : Storm surge inundati...,isaias,"['RT', 'NEW', 'NHCAtlantic', 'Storm', 'surge',...",None
30084,2020-08-03 17:41:11,800,962419303604776968,foxcata7,Not Specified,fema: RT @NHC_Atlantic: Storm surge inundation...,isaias,"['fema', 'RT', 'NHCAtlantic', 'Storm', 'surge'...",None
30085,2020-08-03 17:38:53,800,9309082,huttson,Not Specified,‘INUNDATION: Art and Climate Change in the Pac...,NoTags,"['‘INUNDATION', 'Art', 'Climate', 'Change', 'P...",None
30086,2020-08-03 17:33:13,800,1059455606564798465,sismosvenezuela,Not Specified,#NHC_Atlantic: Storm surge inundation of 3-5 f...,"nhc_atlantic, isaias","['NHCAtlantic', 'Storm', 'surge', 'inundation'...",None
